## Chat GPT 4 Build

In [20]:
from collections import Counter, defaultdict

# Load the content of the provided text files
with open("five_letter_words.txt", "r") as file:
    five_letter_words = [word.strip().upper() for word in file.readlines()]

with open("OldWords.txt", "r") as file:
    OldWords = [word.strip().upper() for word in file.readlines()]

def remove_words(five_letter_words, OldWords):
    """Removes all words from `five_letter_words` that are also in `OldWords`."""
    return [word for word in five_letter_words if word not in OldWords]

five_letter_words = remove_words(five_letter_words, OldWords)

def count_letter_frequency(words_list):
    """Counts the frequency of each letter in the given list of words."""
    letter_frequency = Counter("".join(words_list))
    print(letter_frequency)
    return sorted(letter_frequency.items(), key=lambda x: x[1], reverse=True)

top_common_letters = count_letter_frequency(five_letter_words)

def filter_words_based_on_letters(words_list, top_common_letters, num_letters=5):
    """Filters the words based on the most common letters and extracts unique letters."""
    filtered_words = words_list.copy()
    for letter, _ in top_common_letters[:num_letters]:
        filtered_words = [word for word in filtered_words if letter in word]
    
    unique_letters = set("".join(filtered_words))
    return filtered_words, list(unique_letters)

filtered_words, unique_letters = filter_words_based_on_letters(five_letter_words, top_common_letters)

def most_common_combinations(words_list, starting_letters, top_n=5):
    """Finds the most common two-letter combinations in the given list of words."""
    two_letter_combinations = Counter()
    for word in words_list:
        for i in range(len(word)):
            for j in range(i+1, len(word)):
                if word[i] in starting_letters and word[j] in starting_letters:
                    combo = tuple(sorted([word[i], word[j]]))
                    two_letter_combinations[combo] += 1
    return two_letter_combinations.most_common(top_n)


def filter_words_by_locked_positions(words_list, locked_positions):
    """Filters words that have the locked letters in the locked positions."""
    for position, letter in locked_positions.items():
        words_list = [word for word in words_list if len(word) >= position and word[position-1] == letter]
    return words_list

def filter_words_by_excluded_positions(words_list, excluded_positions):
    """Filters out words that have the specified letters in the excluded positions."""
    for position, letter in excluded_positions.items():
        words_list = [word for word in words_list if len(word) < position or word[position-1] != letter]
    return words_list


def filter_words_based_on_parameters_updated(words_list, letters_included, letters_not_included, locked_positions, excluded_positions):
    """Filters the word list based on the specified parameters."""
    # Filter by letters included
    for letter in letters_included:
        words_list = exclude_words_with_letters(words_list, letters_not_included)
    
    # Filter by letters not included
    words_list = exclude_words_with_letters(words_list, letters_not_included)
    
    # Filter by locked positions
    words_list = filter_words_by_locked_positions(words_list, locked_positions)
    
    # Filter by excluded positions
    words_list = filter_words_by_excluded_positions(words_list, excluded_positions)
    
    return words_list

# Renaming and defining the function for excluding words with certain letters
def exclude_words_with_letters(words_list, excluded_letters):
    """Excludes words that contain any of the letters marked as incorrect by the user."""
    for letter in excluded_letters:
        words_list = [word for word in words_list if letter not in word]
    return words_list

def exclude_words_with_letters(words_list, excluded_letters):
    """Excludes words that contain any of the letters marked as incorrect by the user."""
    for letter in excluded_letters:
        words_list = [word for word in words_list if letter not in word]
    return words_list


Counter({'S': 7102, 'E': 7002, 'A': 6743, 'O': 4909, 'R': 4365, 'I': 4146, 'L': 3526, 'T': 3419, 'N': 3282, 'U': 2762, 'D': 2596, 'P': 2304, 'M': 2297, 'Y': 2257, 'C': 2061, 'H': 1836, 'B': 1745, 'G': 1742, 'K': 1665, 'F': 1161, 'W': 1059, 'V': 750, 'Z': 485, 'J': 333, 'X': 309, 'Q': 134})


## Recomendations for next word

In [21]:
def compute_letter_scores(words_list):
    """Computes the frequency scores for each letter in the words list."""
    letter_frequency = Counter("".join(words_list))
    total_letters = sum(letter_frequency.values())
        
    # Normalize the letter scores
    letter_scores = {letter: freq for letter, freq in letter_frequency.items()}
    #letter_scores = {letter: freq / total_letters for letter, freq in letter_frequency.items()}
    
    return letter_scores

def compute_combination_scores(words_list, unique_letters):
    """Computes the scores for two-letter combinations in the words list."""
    two_letter_combinations = most_common_combinations(words_list, unique_letters, top_n=None)
    total_combinations = sum([count for _, count in two_letter_combinations])
    
    # Normalize the combination scores
    combination_scores = {combo: count / total_combinations for combo, count in two_letter_combinations}
    return combination_scores

def top_n_recommended_words_with_scores(words_list, n=20, repeat_multiplier=0.5):
    """Recommends the top n optimal words based on letter and combination scores and returns the breakdown."""
    letter_scores = compute_letter_scores(words_list)
    combination_scores = compute_combination_scores(words_list, unique_letters)
    
    letter_frequencies = dict(count_letter_frequency(words_list))
    unique_letter_scores = [sum(letter_frequencies[letter] for letter in set(word)) for word in words_list]
    
    total_scores = [ls + cs + uls for ls, cs, uls in zip(letter_scores, combination_scores, unique_letter_scores)]
    
    sorted_indices = sorted(range(len(words_list)), key=lambda k: total_scores[k], reverse=True)
    
    top_words = [words_list[i] for i in sorted_indices[:n]]
    top_letter_scores = [letter_scores[i] for i in sorted_indices[:n]]
    top_combination_scores = [combination_scores[i] for i in sorted_indices[:n]]
    top_unique_letter_scores = [unique_letter_scores[i] for i in sorted_indices[:n]]
    top_total_scores = [total_scores[i] for i in sorted_indices[:n]]
    
    return top_words, top_total_scores, top_letter_scores, top_combination_scores, top_unique_letter_scores



'''
def top_n_recommended_words_with_scores(words_list, top_common_letters, unique_letters, n=20, repeat_multiplier=0.5):
    """Recommends the top n optimal words based on letter and combination scores and returns the breakdown."""
    letter_scores = compute_letter_scores(filtered_words)
    combination_scores = compute_combination_scores(words_list, unique_letters)
    
    repeat_multiplier = float(repeat_multiplier_entry.get()) #Sets the multiplyer via the GUI

    word_scores = {}
    word_letter_scores = {}
    word_combination_scores = {}
    word_unique_letter_scores = {}
    for word in words_list:
        # Letter score
        word_letter_score = sum([letter_scores.get(letter, 0) for letter in word])
        word_letter_scores[word] = round(word_letter_score, 2)
        
        # Unique letter score
        unique_letters_in_word = set(word)
        word_unique_letter_score = sum([letter_scores.get(letter, 0) for letter in unique_letters_in_word])
        word_unique_letter_scores[word] = round(word_unique_letter_score, 2)
        
        # Combination score
        word_combination_score = 0
        for i in range(len(word)):
            for j in range(i+1, len(word)):
                combo = tuple(sorted([word[i], word[j]]))
                word_combination_score += combination_scores.get(combo, 0)
        word_combination_scores[word] = round(word_combination_score, 2)
        
        # Apply the multiplier if the word has repeated letters
        multiplier = repeat_multiplier if len(word) > len(set(word)) else 1.0
                
        # Total score
        total_score = (word_letter_score * multiplier) + word_combination_score + word_unique_letter_score
        word_scores[word] = round(total_score, 2)
        # Get the top n words with the highest unique letter scores, with total score as a tiebreaker
    sorted_words = sorted(word_scores, key=lambda w: (word_unique_letter_scores[w], word_scores[w]), reverse=True)[:n]
    '''

    #headers = ["Word", "Total Score", "Letter Score", "Combination Score", "Unique Letter Score"]
    #return sorted_words, [word_scores[word] for word in sorted_words], [word_letter_scores[word] for word in sorted_words], [word_combination_scores[word] for word in sorted_words], [word_unique_letter_scores[word] for word in sorted_words] 

'\ndef top_n_recommended_words_with_scores(words_list, top_common_letters, unique_letters, n=20, repeat_multiplier=0.5):\n    """Recommends the top n optimal words based on letter and combination scores and returns the breakdown."""\n    letter_scores = compute_letter_scores(filtered_words)\n    combination_scores = compute_combination_scores(words_list, unique_letters)\n    \n    repeat_multiplier = float(repeat_multiplier_entry.get()) #Sets the multiplyer via the GUI\n\n    word_scores = {}\n    word_letter_scores = {}\n    word_combination_scores = {}\n    word_unique_letter_scores = {}\n    for word in words_list:\n        # Letter score\n        word_letter_score = sum([letter_scores.get(letter, 0) for letter in word])\n        word_letter_scores[word] = round(word_letter_score, 2)\n        \n        # Unique letter score\n        unique_letters_in_word = set(word)\n        word_unique_letter_score = sum([letter_scores.get(letter, 0) for letter in unique_letters_in_word])\n       

## Graphical interphase

In [22]:
import tkinter as tk
from tkinter import StringVar, ttk
from functools import partial
from tkinter import ttk, Text, Scrollbar

# Initialize the main window
root = tk.Tk()
root.title("Wordle Solver")

# Variables to store user's choices
letters_included = []
letters_not_included = []
locked_positions = {}
excluded_positions = {}
submitted_word = "" 
letter_states = {}  # {position: "locked"/"excluded"/"wrong_place"}


# Command functions for buttons

# Define a function to reset the button color
def reset_button_color(button):
    button.config(bg='SystemButtonFace')

# Modify the existing command functions to reset the colors of the other buttons
def create_lock_command(lock_btn, exclude_btn, wrong_place_btn, position, letters_states):
    def command():
        letters_states[position]['locked'] = True
        letters_states[position]['excluded'] = False
        letters_states[position]['wrong_place'] = False

        lock_btn.config(bg="green")
        exclude_btn.config(bg="SystemButtonFace")
        wrong_place_btn.config(bg="SystemButtonFace")
    return command

def create_exclude_command(lock_btn, exclude_btn, wrong_place_btn, position, letters_states):
    def command():
        letters_states[position]['locked'] = False
        letters_states[position]['excluded'] = True
        letters_states[position]['wrong_place'] = False

        lock_btn.config(bg="SystemButtonFace")
        exclude_btn.config(bg="dark grey")
        wrong_place_btn.config(bg="SystemButtonFace")
    return command

def create_wrong_place_command(lock_btn, exclude_btn, wrong_place_btn, position, letters_states):
    def command():
        letters_states[position]['locked'] = False
        letters_states[position]['excluded'] = False
        letters_states[position]['wrong_place'] = True

        lock_btn.config(bg="SystemButtonFace")
        exclude_btn.config(bg="SystemButtonFace")
        wrong_place_btn.config(bg="yellow")
    return command





def lock_in_place(button, letter, position):
    global locked_positions  # Declare the variable as global
    locked_positions[position] = letter
    #button.config(state=tk.DISABLED)
    button.config(bg='green')

    # Add the print statements here
    print("After locking a letter:")
    print("Letters not included:", letters_not_included)
    print("Locked positions:", locked_positions)
    print("Excluded positions:", excluded_positions)

def do_not_include(button, letter):
    global letters_not_included  # Declare the variable as global
    letters_not_included.append(letter)
    #button.config(state=tk.DISABLED)
    button.config(bg='darkgray')

    # Add the print statements here
    print("After excluding a letter:")
    print("Letters not included:", letters_not_included)
    print("Locked positions:", locked_positions)
    print("Excluded positions:", excluded_positions)

def wrong_place(button, letter, position):
    global letters_not_included, excluded_positions  # Declare the variables as global
    excluded_positions[position] = letter
    letters_not_included.append(letter)
    #button.config(state=tk.DISABLED)
    button.config(bg='yellow')

    # Add the print statements here
    print("After marking a letter as in the wrong place:")
    print("Letters not included:", letters_not_included)
    print("Locked positions:", locked_positions)
    print("Excluded positions:", excluded_positions)


def display_buttons(position):
    if letter_states[position] == "locked":
        # Change the appearance of the lock button, reset others
        ...
    elif letter_states[position] == "excluded":
        # Change the appearance of the exclude button, reset others
        ...
    elif letter_states[position] == "wrong_place":
        # Change the appearance of the wrong place button, reset others
        ...

def exclude_letter(button, letter, position):
    # Logic for marking the letter as excluded
    # Update the global variable
    letters_not_included.append(letter)
    # Update the UI, if necessary
    button.config(bg='red')  # As an example, change the background to red

def mark_wrong_place(button, letter, position):
    # Logic for marking the letter as in the wrong place
    excluded_positions[position] = letter
    # Update the UI
    button.config(bg='yellow')  # Change the background to yellow, as an example




from functools import partial

def display_letters():

    global submitted_word  # Declare it as global
    submitted_word = word_entry.get().upper()  # Retrieve the word from the entry widget
    solve_wordle()
    # Initialize the local state for each letter
    letters_states = [{} for _ in submitted_word]

    print("display_letters function called!")
    for widget in root.winfo_children():
        widget.grid_forget()  # Clear the previous widgets

    # Initialize the local state for each letter
    letters_states = {i: {'locked': False, 'excluded': False, 'wrong_place': False} for i in range(5)}
    
    def solve_wordle_local():
        global locked_positions, excluded_positions, letters_not_included
        locked_positions = {}
        excluded_positions = {}
        letters_not_included = []

        for position, state in letters_states.items():
            if state['locked']:
                locked_positions[position+1] = submitted_word[position]
            elif state['excluded']:
                letters_not_included.append(submitted_word[position])
            elif state['wrong_place']:
                excluded_positions[position+1] = submitted_word[position]

        solve_wordle()

    solve_btn.config(command=solve_wordle_local)


    # Create all buttons first
    lock_buttons = []
    exclude_buttons = []
    wrong_place_buttons = []

    for i, letter in enumerate(submitted_word):
        ttk.Label(root, text=letter, width=5).grid(row=2, column=i+1)

        lock_button = tk.Button(root, text="Lock")
        lock_button.grid(row=2, column=i+1)
        lock_buttons.append(lock_button)

        exclude_button = tk.Button(root, text="Exclude")
        exclude_button.grid(row=3, column=i+1)
        exclude_buttons.append(exclude_button)

        wrong_place_button = tk.Button(root, text="Wrong Place")
        wrong_place_button.grid(row=4, column=i+1)
        wrong_place_buttons.append(wrong_place_button)

    # Now configure their commands
    for i, letter in enumerate(submitted_word):
        lock_buttons[i].config(command=create_lock_command(lock_buttons[i], exclude_buttons[i], wrong_place_buttons[i], i, letters_states))
        exclude_buttons[i].config(command=create_exclude_command(lock_buttons[i], exclude_buttons[i], wrong_place_buttons[i], i, letters_states))
        wrong_place_buttons[i].config(command=create_wrong_place_command(lock_buttons[i], exclude_buttons[i], wrong_place_buttons[i], i, letters_states))

    #Row 0-1
    word_entry.grid(row=0, column=1, padx=10, pady=10)
    submit_btn.grid(row=0, column=2, padx=10, pady=10)
    solve_btn.grid(row=0, column=3, columnspan=5, pady=10)

    repeat_multiplier_label.grid(row=1, column=1, padx=10, pady=10, sticky='e')
    repeat_multiplier_entry.grid(row=1, column=2, padx=10, pady=10)

    #Row 7
    output_text.grid(row=7, column=0, columnspan=6, padx=10, pady=10)
    scrollbar.grid(row=7, column=6, sticky='ns')
  
    



def update_global_state(letters_states):
    global letters_not_included, locked_positions, excluded_positions
    
    # Initialize the global variables to empty
    locked_positions = {}
    excluded_positions = {}
    letters_not_included = []

    # Loop through the letters_states list and update global variables
    for i, state in enumerate(letters_states):
        if state == "locked":
            locked_positions[i + 1] = submitted_word[i]  # The +1 is because the position starts from 1
        elif state == "excluded":
            letters_not_included.append(submitted_word[i])
        elif state == "wrong_place":
            excluded_positions[i + 1] = submitted_word[i]

    # Debug
    print("Updated Global State:")
    print("locked_positions:", locked_positions)
    print("excluded_positions:", excluded_positions)
    print("letters_not_included:", letters_not_included)




def solve_wordle():
    word = submitted_word
    global letters_included, letters_not_included, locked_positions, excluded_positions
    
    # Compute the top common letters from the entire word list
    letter_freq_str_all = compute_letter_scores(filtered_words)
    top_common_letters = [letter for letter, _ in sorted(letter_freq_str_all.items(), key=lambda item: item[1], reverse=True)]

    # Get the word from the entry
    word = submitted_word.upper()
    
    # Filter the words
    filtered_words_updated = filter_words_based_on_parameters_updated(five_letter_words, letters_included, letters_not_included, locked_positions, excluded_positions)
    print("Filtered words after processing:", filtered_words_updated[:10])

    # Recommend the next word
    top_words, top_total_scores, top_letter_scores, top_combination_scores, top_unique_letter_scores = top_n_recommended_words_with_scores(filtered_words_updated, top_common_letters, unique_letters)
    print("")

    # Prepare the output to be displayed
    top_common_letters = count_letter_frequency(filtered_words_updated)
    top_common_letters_list = [f"{letter}: {freq}" for letter, freq in top_common_letters]
    print(top_common_letters_list)

    output_content = []    
    header = "      Tot  Lett Combo Uniqe"
    output_content = [header]

    word_rows  = [f"{word} {total_score}, {letter_score}, {combination_scores}, {unique_letter_scores}" for word, total_score, letter_score, combination_scores, unique_letter_scores  in zip(top_words, top_total_scores, top_letter_scores, top_combination_scores, top_unique_letter_scores)]
    print(word_rows)
    # Append the word rows to the output content
    output_content.extend(word_rows)
    output_content.append('\n')  # Adds a new line
    output_content.extend(top_common_letters_list)

    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, '\n'.join(output_content))


# Function to clear the output and reset the input
def clear_and_reset():
    global submitted_word
    submitted_word = ""  # Reset the submitted word
    for widget in root.winfo_children():
        widget.grid_forget()
    create_initial_ui()

def submit_word():
    global submitted_word
    submitted_word = next_word_entry.get().upper()
    display_letters()
  

#!!
letter_frequencies = dict(count_letter_frequency(filtered_words))
unique_letter_scores = [sum(letter_frequencies[letter] for letter in set(word)) for word in filtered_words]


# Button for Submit Word
submit_btn = tk.Button(root, text="Submit Word", command=display_letters)
submit_btn.grid(row=0, column=2, padx=10, pady=10)

# Button to Solve Wordle
solve_btn = tk.Button(root, text="Solve Wordle", command=solve_wordle)
solve_btn.grid(row=0, column=3, columnspan=5, pady=10)

# Text widget to display multiple lines of output
output_text = Text(root, width=50, height=22)
output_text.grid(row=7, column=0, columnspan=6, padx=10, pady=10)
scrollbar = Scrollbar(root, command=output_text.yview)
scrollbar.grid(row=7, column=6, sticky='nsew')
output_text.config(yscrollcommand=scrollbar.set)

# Entry for the next word
word_entry = ttk.Entry(root, width=15)
word_entry.grid(row=0, column=1, padx=10, pady=10)
word_entry.insert(0, "AEROS") #Suggested starting word

# Create a label for clarity
repeat_multiplier_label = ttk.Label(root, text="Repeat Multiplier:")
repeat_multiplier_label.grid(row=6, column=3, padx=10, pady=10, sticky='e')  # Adjust the row and column as needed

# The actual entry
repeat_multiplier_entry = ttk.Entry(root, width=5)
repeat_multiplier_entry.grid(row=6, column=4, padx=10, pady=10)
repeat_multiplier_entry.insert(0, "0.5")  # Default value


root.mainloop()


Counter({'A': 3, 'E': 3, 'R': 3, 'O': 3, 'S': 3})
Filtered words after processing: ['AAHED', 'AALII', 'AAPAS', 'AARGH', 'AARTI', 'ABACA', 'ABACI', 'ABACS', 'ABAFT', 'ABAHT']
Counter({'S': 7102, 'E': 7002, 'A': 6743, 'O': 4909, 'R': 4365, 'I': 4146, 'L': 3526, 'T': 3419, 'N': 3282, 'U': 2762, 'D': 2596, 'P': 2304, 'M': 2297, 'Y': 2257, 'C': 2061, 'H': 1836, 'B': 1745, 'G': 1742, 'K': 1665, 'F': 1161, 'W': 1059, 'V': 750, 'Z': 485, 'J': 333, 'X': 309, 'Q': 134})


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\JonasCarlsson\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\JonasCarlsson\AppData\Local\Temp\ipykernel_19784\2329719846.py", line 134, in display_letters
    solve_wordle()
  File "C:\Users\JonasCarlsson\AppData\Local\Temp\ipykernel_19784\2329719846.py", line 248, in solve_wordle
    top_words, top_total_scores, top_letter_scores, top_combination_scores, top_unique_letter_scores = top_n_recommended_words_with_scores(filtered_words_updated, top_common_letters, unique_letters)
                                                                                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\JonasCarlsson\AppData\Local\Temp\ipykernel_19784\1836258508.py", line 29, in top_n_recommended_words_with_

KeyboardInterrupt: 